In [1]:
from datasets import Dataset
from ragas import evaluate
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.metrics import AnswerCorrectness, AnswerRelevancy, ContextPrecision, ContextRecall, Faithfulness
import time

c:\Users\wills\anaconda3\envs\llm_rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Setup Evaluation Dataset

In [2]:
import pandas as pd

In [8]:
df = pd.read_csv("SoftEng Evaluation Data Final 2.csv", encoding="UTF-8", sep=";")
df.head()

,file_name,question,ground_truth,answer,context_1,context_2,context_3
0,Atribut_BGA.txt,Sebutkan 9 atribut yang termasuk dalam Binus G...,9 atribut dalam Binus Graduate Attributes (BGA...,9 atribut dalam Binus Graduate Attributes (BGA...,BINUS Graduate Attribute (BGA) merupakan kumpu...,Terdapat 9 (sembilan) atribut atau keterampila...,1.Global Standard Technical Competencies (GSTC...
1,Cara_Akses_Artikel_Binus_Support.txt,Bagaimana langkah-langkah mengakses Microsite ...,"Setelah login, klik ikon Apps Launcher yang be...","Setelah login ke BINUSMAYA, langkahnya adalah ...",Langkah 1: Login ke BINUSMAYA. Buka peramban (...,Langkah 3: Pilih Aplikasi BINUS Support. Setel...,Universitas Bina Nusantara (BINUS) menyediakan...
2,Cara_Mahasiswa_Mengetahui_Gelar_Kelulusan.txt,Di bagian mana mahasiswa dapat menemukan infor...,"Setelah berada di halaman profil, mahasiswa ha...",Informasi gelar kelulusan (Degree Title) dapat...,Langkah 3: Navigasi ke Halaman Profil. Setelah...,"Pada halaman ini, cari kolom atau bagian yang ...",Mahasiswa Universitas Bina Nusantara (BINUS) d...
3,ComServ.txt,Apa saja persyaratan spesifik yang harus dipen...,Persyaratan untuk Video Based Learning adalah:...,Untuk Community Service jenis Video Based Lear...,5. Video Based Learning. Pembuatan video eduka...,6. Massive Open Online Course (MOOC). Program ...,Community Services (ComServ) adalah kegiatan s...
4,Daftar_Rektor_Binus.txt,Siapa rektor yang menjabat pada periode 2005–2...,Rektor yang menjabat pada periode 2005–2009 ad...,Rektor yang menjabat pada periode 2005–2009 ad...,"2. Prof. Gerardus Polla, M.App.Sc. (Periode 20...","Di era ini juga, BINUS memperluas cakupan prog...","Dr. Ir. Th. Widia Soerjaningsih, M.M. (Periode..."


In [9]:
def combine_contexts(row):
    contexts = []
    for col in df.columns:
        if col.startswith("context_"):
            content = str(row[col])
            if content and content.lower() != "nan":
                contexts.append(content)
    return contexts

In [10]:
df["contexts"] = df.apply(combine_contexts, axis=1)
df.head()

,file_name,question,ground_truth,answer,context_1,context_2,context_3,contexts
0,Atribut_BGA.txt,Sebutkan 9 atribut yang termasuk dalam Binus G...,9 atribut dalam Binus Graduate Attributes (BGA...,9 atribut dalam Binus Graduate Attributes (BGA...,BINUS Graduate Attribute (BGA) merupakan kumpu...,Terdapat 9 (sembilan) atribut atau keterampila...,1.Global Standard Technical Competencies (GSTC...,[BINUS Graduate Attribute (BGA) merupakan kump...
1,Cara_Akses_Artikel_Binus_Support.txt,Bagaimana langkah-langkah mengakses Microsite ...,"Setelah login, klik ikon Apps Launcher yang be...","Setelah login ke BINUSMAYA, langkahnya adalah ...",Langkah 1: Login ke BINUSMAYA. Buka peramban (...,Langkah 3: Pilih Aplikasi BINUS Support. Setel...,Universitas Bina Nusantara (BINUS) menyediakan...,[Langkah 1: Login ke BINUSMAYA. Buka peramban ...
2,Cara_Mahasiswa_Mengetahui_Gelar_Kelulusan.txt,Di bagian mana mahasiswa dapat menemukan infor...,"Setelah berada di halaman profil, mahasiswa ha...",Informasi gelar kelulusan (Degree Title) dapat...,Langkah 3: Navigasi ke Halaman Profil. Setelah...,"Pada halaman ini, cari kolom atau bagian yang ...",Mahasiswa Universitas Bina Nusantara (BINUS) d...,[Langkah 3: Navigasi ke Halaman Profil. Setela...
3,ComServ.txt,Apa saja persyaratan spesifik yang harus dipen...,Persyaratan untuk Video Based Learning adalah:...,Untuk Community Service jenis Video Based Lear...,5. Video Based Learning. Pembuatan video eduka...,6. Massive Open Online Course (MOOC). Program ...,Community Services (ComServ) adalah kegiatan s...,[5. Video Based Learning. Pembuatan video eduk...
4,Daftar_Rektor_Binus.txt,Siapa rektor yang menjabat pada periode 2005–2...,Rektor yang menjabat pada periode 2005–2009 ad...,Rektor yang menjabat pada periode 2005–2009 ad...,"2. Prof. Gerardus Polla, M.App.Sc. (Periode 20...","Di era ini juga, BINUS memperluas cakupan prog...","Dr. Ir. Th. Widia Soerjaningsih, M.M. (Periode...","[2. Prof. Gerardus Polla, M.App.Sc. (Periode 2..."


In [11]:
ragas_df = df[["question", "answer", "contexts", "ground_truth"]]
evaluation_dataset = Dataset.from_pandas(ragas_df)
evaluation_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 21
})

In [12]:
print("Format Dataset RAGAS:")
print(evaluation_dataset[0])

Format Dataset RAGAS:
{'question': 'Sebutkan 9 atribut yang termasuk dalam Binus Graduate a (BGA) dan apa landasan utama pengembangannya?', 'answer': '9 atribut dalam Binus Graduate Attributes (BGA) adalah Global Standard Technical Competencies, Digital & Technology Fluency, Applied Management Skills, Critical & Creative Thinking, Adaptability, Initiative, Growth Mindset, Collaboration, dan Social Awareness. Landasan utama pengembangannya adalah Binusian Values (S-P-I-R-I-T) untuk memperkuat karakter mahasiswa.', 'contexts': ['BINUS Graduate Attribute (BGA) merupakan kumpulan atribut terkait kualitas atau keterampilan yang perlu dimiliki dan dikembangkan oleh para lulusan BINUS University selama menjalani perkuliahan di BINUS University melalui proses Educate, Enrich dan Empower guna mempersiapkan diri menghadapi masa depan. Oleh karena itu, seluruh mahasiswa perlu mengembangkan BINUS Graduate Attributes untuk memperlengkapi dan mempersiapkan agar kelak dapat menjalani secara optimal 3

# Setup Gemini LLM & Embeddings

In [ ]:
google_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key="GEMINI-API-KEY"
)
ragas_llm = LangchainLLMWrapper(google_llm)

C:\Users\wills\AppData\Local\Temp\ipykernel_19036\3169411544.py:6: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  ragas_llm = LangchainLLMWrapper(google_llm)


# Evaluation

In [14]:
import nest_asyncio
from ragas.run_config import RunConfig

In [15]:
nest_asyncio.apply()

my_run_config = RunConfig(
    max_workers=1,
    timeout=300,
    max_retries=3,
    max_wait=5
)

In [ ]:
class SlowEmbeddings(GoogleGenerativeAIEmbeddings):
    def embed_documents(self, texts, **kwargs):
        time.sleep(4)
        return super().embed_documents(texts, **kwargs)

    def embed_query(self, text, **kwargs):
        time.sleep(4)
        return super().embed_query(text, **kwargs)

google_embeddings = SlowEmbeddings(
    model="models/text-embedding-004",
    google_api_key="GEMINI-API-KEY"
)

ragas_embeddings = LangchainEmbeddingsWrapper(google_embeddings)

C:\Users\wills\AppData\Local\Temp\ipykernel_19036\687850375.py:15: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  ragas_embeddings = LangchainEmbeddingsWrapper(google_embeddings)


In [17]:
scores = evaluate(
    evaluation_dataset,
    metrics=[
        ContextPrecision(llm=ragas_llm),
        ContextRecall(llm=ragas_llm),
        Faithfulness(llm=ragas_llm),
        AnswerCorrectness(llm=ragas_llm, embeddings=ragas_embeddings),
        AnswerRelevancy(llm=ragas_llm, embeddings=ragas_embeddings)
    ],
    run_config=my_run_config
)

print("HASIL EVALUASI:")
print(scores)

Evaluating: 100%|██████████| 105/105 [34:21<00:00, 19.63s/it]


HASIL EVALUASI:
{'context_precision': 0.6627, 'context_recall': 0.9286, 'faithfulness': 0.9563, 'answer_correctness': 0.8549, 'answer_relevancy': 0.9031}
